In [11]:
import cv2
import numpy as np
import glob
import yaml

# Configuration ---------------------------------------------------------------
CHECKERBOARD_DIM = (7, 9)          # Number of checkerboard corners in rows and columns
SQUARE_DIM = 0.015                # Size of each square on the checkerboard in meters
LEFT_IMAGE_PATH = r"Dataset_6/left/*.png"  # Path for left camera calibration images
RIGHT_IMAGE_PATH = r"Dataset_6/right/*.png"  # Path for right camera calibration images
TARGET_IMAGE_SIZE = (1280, 720)   # Expected image resolution (width, height)

# Prepare 3D points for the checkerboard (object points)
object_points = np.zeros((CHECKERBOARD_DIM[0] * CHECKERBOARD_DIM[1], 3), np.float32)
object_points[:, :2] = np.mgrid[0:CHECKERBOARD_DIM[0], 0:CHECKERBOARD_DIM[1]].T.reshape(-1, 2) * SQUARE_DIM

def calibrate_single_camera(image_paths, camera_label):
    """Detect checkerboard corners in images and compute camera intrinsic parameters."""
    object_points_list, image_points_list = [], []
    valid_images, invalid_images = [], []
    image_files = sorted(glob.glob(image_paths))
    if not image_files:
        raise ValueError(f"No images found at {image_paths}!")

    print(f"\n=== Processing {camera_label} ===")
    print(f"Total images found: {len(image_files)}")

    for idx, image_file in enumerate(image_files, 1):
        image = cv2.imread(image_file)
        if image is None:
            invalid_images.append(image_file)
            continue

        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        found_corners, corners = cv2.findChessboardCorners(
            grayscale_image, CHECKERBOARD_DIM,
            cv2.CALIB_CB_ADAPTIVE_THRESH +
            cv2.CALIB_CB_FAST_CHECK +
            cv2.CALIB_CB_NORMALIZE_IMAGE
        )

        if found_corners:
            refined_corners = cv2.cornerSubPix(
                grayscale_image, corners, (11, 11), (-1, -1),
                (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
            )
            object_points_list.append(object_points)
            image_points_list.append(refined_corners)
            valid_images.append(image_file)
        else:
            invalid_images.append(image_file)

    if not object_points_list:
        raise ValueError(f"No valid checkerboard images found for {camera_label}!")

    retval, camera_matrix, distortion_coeffs, rotation_vecs, translation_vecs = cv2.calibrateCamera(
        object_points_list, image_points_list, grayscale_image.shape[::-1], None, None,
        flags=cv2.CALIB_ZERO_TANGENT_DIST +
              cv2.CALIB_FIX_K3 +
              cv2.CALIB_FIX_K4 +
              cv2.CALIB_FIX_K5
    )

    avg_reprojection_error = 0
    for i in range(len(object_points_list)):
        projected_points, _ = cv2.projectPoints(object_points_list[i], rotation_vecs[i], translation_vecs[i], camera_matrix, distortion_coeffs)
        error = cv2.norm(image_points_list[i], projected_points, cv2.NORM_L2) / len(projected_points)
        avg_reprojection_error += error
    avg_reprojection_error /= len(object_points_list)

    return camera_matrix, distortion_coeffs, object_points_list, image_points_list, avg_reprojection_error, image_files

# Stereo calibration pipeline -----------------------------------------------
try:
    print("Starting stereo calibration...")

    left_camera_matrix, left_distortion, objpoints_left, imgpoints_left, left_error, _ = \
        calibrate_single_camera(LEFT_IMAGE_PATH, "Left Camera")
    right_camera_matrix, right_distortion, objpoints_right, imgpoints_right, right_error, _ = \
        calibrate_single_camera(RIGHT_IMAGE_PATH, "Right Camera")

    calibration_flags = cv2.CALIB_FIX_INTRINSIC + cv2.CALIB_USE_INTRINSIC_GUESS
    calibration_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5)

    retval, K1, D1, K2, D2, rotation_matrix, translation_vector, essential_matrix, fundamental_matrix = cv2.stereoCalibrate(
        objpoints_left, imgpoints_left, imgpoints_right,
        left_camera_matrix, left_distortion, right_camera_matrix, right_distortion,
        TARGET_IMAGE_SIZE,
        criteria=calibration_criteria, flags=calibration_flags
    )

    # Removed stereoRectify and related variables
    stereo_calibration_data = {
        'image_size': TARGET_IMAGE_SIZE,
        'K_left': left_camera_matrix, 'D_left': left_distortion,
        'K_right': right_camera_matrix, 'D_right': right_distortion,
        'R': rotation_matrix, 'T': translation_vector,
        'E': essential_matrix, 'F': fundamental_matrix,
        'error_left': left_error,
        'error_right': right_error,
        'stereo_error': retval
    }

    # Saving results to YAML (handling arrays and tuples)
    with open(r'stereo_calibration_results.yaml', 'w') as file:
        yaml.dump({
            k: (v.tolist() if isinstance(v, np.ndarray) else
                [list(x) if isinstance(x, tuple) else x for x in v] if isinstance(v, list) else
                (list(v) if isinstance(v, tuple) else v))  # Handling tuples
            for k, v in stereo_calibration_data.items()
        }, file)

    print("Stereo calibration successful.")
    print("Results saved to stereo_calibration_results.yaml")

    with open(r'stereo_calibration_results.yaml', 'r') as file:
        loaded_data = yaml.safe_load(file)

    def print_matrix(name, matrix):
        print(f"\n{name}:")
        for row in matrix:
            print("  [", ", ".join(f"{x:.8f}" for x in row), "]")

    print("=== Left Camera Calibration Results ===")
    print(f"Mean Reprojection Error: {loaded_data['error_left']:.5f} px")
    print_matrix("Camera Matrix", loaded_data['K_left'])
    print("Distortion Coefficients:", np.array(loaded_data['D_left']).ravel())

    print("\n=== Right Camera Calibration Results ===")
    print(f"Mean Reprojection Error: {loaded_data['error_right']:.5f} px")
    print_matrix("Camera Matrix", loaded_data['K_right'])
    print("Distortion Coefficients:", np.array(loaded_data['D_right']).ravel())

    print("\n=== Stereo Calibration Results ===")
    print(f"Stereo Reprojection Error: {loaded_data['stereo_error']:.5f}")
    print_matrix("Rotation Matrix (R)", loaded_data['R'])
    print_matrix("Translation Vector (T)", loaded_data['T'])
    print("Image Size:", tuple(loaded_data['image_size']))

except Exception as ex:
    print(f"Calibration failed: {ex}")


Starting stereo calibration...

=== Processing Left Camera ===
Total images found: 57

=== Processing Right Camera ===
Total images found: 57
Stereo calibration successful.
Results saved to stereo_calibration_results.yaml
=== Left Camera Calibration Results ===
Mean Reprojection Error: 0.02066 px

Camera Matrix:
  [ 1059.82779022, 0.00000000, 646.18224656 ]
  [ 0.00000000, 1059.39069888, 405.35767342 ]
  [ 0.00000000, 0.00000000, 1.00000000 ]
Distortion Coefficients: [ 0.0335875  -0.38226424  0.          0.          0.        ]

=== Right Camera Calibration Results ===
Mean Reprojection Error: 0.01956 px

Camera Matrix:
  [ 1222.90688624, 0.00000000, 635.64289995 ]
  [ 0.00000000, 1220.45822800, 354.18104589 ]
  [ 0.00000000, 0.00000000, 1.00000000 ]
Distortion Coefficients: [ 0.06259462 -0.86894772  0.          0.          0.        ]

=== Stereo Calibration Results ===
Stereo Reprojection Error: 45.88093

Rotation Matrix (R):
  [ 0.99753504, -0.03532767, 0.06062839 ]
  [ 0.03203489, 